# Ćwiczenie 2 - TIER protocol i tidy data

Celem ćwiczenia jest pobranie i uporządkowanie przydzielonego zbioru danych zgodnie z protokołem TIER i zasadami "tidy data".

- W uporządkowanych danych:
  - Każda zmienna tworzy kolumnę.
  - Każda obserwacja tworzy rząd.
  - Każdy typ jednostki obserwacyjnej tworzy tabelę.

-  [TIER Protocol](https://www.projecttier.org/tier-protocol/tier-protocol-version-history/specifications-3-0/#overview-of-the-documentation/)

*weather.txt* - Dzienne dane pogodowe z Global Historical Climatology Network dla jednej stacji pogodowej (MX17004) w Meksyku przez pięć miesięcy w 2010 roku. Pierwsza kolumna zawiera dane (id, rok, miesiąc,nazwa zmiennych), w pozostałych kolumnach są wartości zmiennych na dany dzien miesiąca (dzień, d1 – d31). Miesiące z mniej niż 31 dni mają strukturalne brakujące wartości dla ostatniego dnia (dni) miesiąca. 


In [1]:
import pandas as pd

### Reading and displaying data

In [2]:
df = pd.read_csv('../Original Data/weather.txt', header=None, names=['data'])
df.head(5)

,data
0,MX000017004195504TMAX 310 I 310 I 310 I ...
1,MX000017004195504TMIN 150 I 150 I 160 I ...
2,MX000017004195504PRCP 0 I 0 I 0 I ...
3,MX000017004195505TMAX 310 I 310 I 310 I ...
4,MX000017004195505TMIN 200 I 160 I 160 I ...


### Preprocessing data

Extracting needed features, creating helper columns

In [3]:
df['id'] = 'MX17004'
df['year_month'] = df['data'].str[11:15] + '-' + df['data'].str[15:17]
df['element'] = df['data'].str[17:21]
df['values'] = df['data'].str[21:]
df['values'] = df['values'].str.replace('O', '', regex=False)
df['values'] = df['values'].str.replace('I', '', regex=False)
df['values'] = df['values'].str.replace('S', '', regex=False)
df['values'] = df['values'].str.split()
df[[f'{day:02d}' for day in range(1, 32)]] = pd.DataFrame(df['values'].tolist(), index=df.index)

### Removing unnecessary columns

In [4]:
df = df.drop(columns=['data', 'values'])
df.head(5)

,id,year_month,element,01,02,03,04,05,06,07,...,22,23,24,25,26,27,28,29,30,31
0,MX17004,1955-04,TMAX,310,310,310,320,330,320,320,...,330,330,330,330,330,330,340,330,320,-9999
1,MX17004,1955-04,TMIN,150,150,160,150,160,160,160,...,170,170,170,180,190,190,170,180,160,-9999
2,MX17004,1955-04,PRCP,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,-9999
3,MX17004,1955-05,TMAX,310,310,310,300,300,300,310,...,330,340,350,330,310,310,320,310,300,290
4,MX17004,1955-05,TMIN,200,160,160,150,150,150,160,...,170,190,190,190,180,160,150,170,150,160


### Melting DataFrame

In [5]:
df_melted = pd.melt(df, id_vars=['id','year_month','element'],
                    value_vars=[f'{day:02d}' for day in range(1, 32)],
                    var_name='day')
df_melted.head(5)

,id,year_month,element,day,value
0,MX17004,1955-04,TMAX,01,310
1,MX17004,1955-04,TMIN,01,150
2,MX17004,1955-04,PRCP,01,0
3,MX17004,1955-05,TMAX,01,310
4,MX17004,1955-05,TMIN,01,200


### Removing missing values '-9999' in data

In [6]:
df_melted = df_melted.loc[df_melted['value'] != '-9999']
df_melted = df_melted.reset_index()
df_melted.head(5)

,index,id,year_month,element,day,value
0,0,MX17004,1955-04,TMAX,01,310
1,1,MX17004,1955-04,TMIN,01,150
2,2,MX17004,1955-04,PRCP,01,0
3,3,MX17004,1955-05,TMAX,01,310
4,4,MX17004,1955-05,TMIN,01,200


### Now we can create date column for indexing

In [7]:
df_melted['date'] = df_melted['year_month'] + df_melted['day']
df_melted['date'] = pd.to_datetime(df_melted['date'], format='%Y-%m%d')
df_melted.head(5)

,index,id,year_month,element,day,value,date
0,0,MX17004,1955-04,TMAX,01,310,1955-04-01
1,1,MX17004,1955-04,TMIN,01,150,1955-04-01
2,2,MX17004,1955-04,PRCP,01,0,1955-04-01
3,3,MX17004,1955-05,TMAX,01,310,1955-05-01
4,4,MX17004,1955-05,TMIN,01,200,1955-05-01


### Pivoting DataFrame and removing rows with 'TMAX' and 'TMIN' NaN values

In [8]:
df_pivot = df_melted.pivot(columns='element', values='value', index=['id', 'date'])
df_pivot = df_pivot.dropna(subset=['TMAX', 'TMIN'])
df_pivot.head(5)

element            PRCP TMAX TMIN
id      date                     
MX17004 1955-04-01    0  310  150
        1955-04-02    0  310  150
        1955-04-03    0  310  160
        1955-04-04    0  320  150
        1955-04-05    0  330  160

### Saving final DataFrame to .csv

In [10]:
df_pivot.to_csv('../Analysis Data/cleaned_weather.csv')